In [1]:
import pickle5 as pickle
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split

In [7]:
actions = [1, 2, 3, 5, 6, 7, 8, 9, 10, 23, 24, 26, 27, 29, 31, 34, 35, 36, 39, 40]

skes_data = []

for a in actions:
    with open("raw_data/A"+ str(a).zfill(3) + ".pkl", "rb") as f:
      data = pickle.load(f)

    skes_data += data
    print(a, len(data), len(skes_data))

1 940 940
2 941 1881
3 938 2819
5 942 3761
6 943 4704
7 944 5648
8 941 6589
9 936 7525
10 941 8466
23 944 9410
24 946 10356
26 947 11303
27 948 12251
29 944 13195
31 944 14139
34 947 15086
35 947 16033
36 946 16979
39 948 17927
40 936 18863


In [5]:
print(len(skes_data))

18863


In [41]:
def get_actor_data(body_data, num_frames):
    joints = np.zeros((num_frames, 75), dtype=np.float32)
    colors = np.ones((num_frames, 1, 25, 2), dtype=np.float32) * np.nan
    joints = body_data['skel_body'].reshape(-1, 75)
    colors[:, 0] = body_data['rgb_body']
    
    return joints, colors

In [42]:
setup = []
camera = []
label = []
replication = []
performer = []
raw_joints = []
raw_colors = []
frame_cnt = []

for sequence in data:
    setup.append(sequence['setup'])
    camera.append(sequence['camera'])
    performer.append(sequence['subject'])
    replication.append(sequence['replication'])
    label.append(sequence['action'])
    frame_cnt.append(sequence['nframes'])
    
    joints, colors = get_actor_data(sequence["bodies_data"][0], sequence['nframes'])
    
    raw_joints.append(joints)
    raw_colors.append(colors)

In [46]:
def seq_translation(skes_joints):
    for idx, ske_joints in enumerate(skes_joints):
        num_frames = ske_joints.shape[0]

        num_bodies = 1 if ske_joints.shape[1] == 75 else 2
        if num_bodies == 2:
            missing_frames_1 = np.where(ske_joints[:, :75].sum(axis=1) == 0)[0]
            missing_frames_2 = np.where(ske_joints[:, 75:].sum(axis=1) == 0)[0]
            cnt1 = len(missing_frames_1)
            cnt2 = len(missing_frames_2)

        i = 0  # get the "real" first frame of actor1
        while i < num_frames:
            if np.any(ske_joints[i, :75] != 0):
                break
            i += 1
        
        origin = np.copy(ske_joints[i, 3:6])  # new origin: joint-2

        for f in range(num_frames):
            if num_bodies == 1:
                ske_joints[f] -= np.tile(origin, 25)
            else:  # for 2 actors
                ske_joints[f] -= np.tile(origin, 50)

        if (num_bodies == 2) and (cnt1 > 0):
            ske_joints[missing_frames_1, :75] = np.zeros((cnt1, 75), dtype=np.float32)

        if (num_bodies == 2) and (cnt2 > 0):
            ske_joints[missing_frames_2, 75:] = np.zeros((cnt2, 75), dtype=np.float32)

        skes_joints[idx] = ske_joints  # Update

    return skes_joints


In [47]:
skes_joints = seq_translation(raw_joints)

In [51]:
np.savetxt("camera.txt", camera, fmt='%d')
np.savetxt("setup.txt", setup, fmt='%d')
np.savetxt("label.txt", label, fmt='%d')
np.savetxt("performer.txt", performer, fmt='%d')
np.savetxt("replication.txt", replication, fmt='%d')
np.savetxt("frame_cnt.txt", frame_cnt, fmt='%d')

In [52]:
with open("raw_denoised_joints.pkl", 'wb') as f:
    pickle.dump(raw_joints, f, pickle.HIGHEST_PROTOCOL)
    
with open("raw_denoised_colors.pkl", 'wb') as f:
    pickle.dump(raw_joints, f, pickle.HIGHEST_PROTOCOL)